### Importación de los módulos necesarios

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pmdarima.arima import ndiffs, nsdiffs
from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from dateutil.relativedelta import relativedelta
from metricas import mae, mape, rmse

import warnings
warnings.simplefilter("ignore")

# Mostramos las versiones de los módulos para posibles reproducciones del código

print('Versión pandas:', pd.__version__)
print('Versión numpy:', np.__version__)
print('Versión matplotlib:', matplotlib.__version__)
print('Version statsmodels', sm.__version__)

Versión pandas: 1.0.5
Versión numpy: 1.19.5
Versión matplotlib: 3.2.2
Version statsmodels 0.12.2


### Lectura del dataframe horario

In [2]:
df = pd.read_csv('Data/datos_horarios_prediccion_electricidad.csv', index_col = 0)

df.index = pd.to_datetime(df.index)

df = df.sort_index()


### Modelo regresión lineal

In [14]:
db = df.dropna()
db = db.drop(columns = ["Prevision_Demanda", "Prevision_Eol_Fotov", "Festivo_Regional"])

X = sm.add_constant(db)
y = X.pop('Spot_electricidad')

model = sm.OLS(y,X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Spot_electricidad   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     9037.
Date:                Sun, 16 Jan 2022   Prob (F-statistic):               0.00
Time:                        17:10:40   Log-Likelihood:            -1.0160e+05
No. Observations:               25536   AIC:                         2.033e+05
Df Residuals:                   25503   BIC:                         2.035e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -13.5580      0.574    -23.619      0.000     -14.683     -12.433
Hueco_Termico        0.0011   1.89e-05     56.910      0.000       0.001       0.001
Lunes               10.2946      0.305     33.705      0.000       9.696      10.893
Martes               8.0837      0.305     26.461      0.000       7.485       8.682
Miercoles            6.7826      0.309     21.969      0.000       6.177       7.388
Jueves               7.2055      0.309     23.281      0.000       6.599       7.812
Viernes              6.2542      0.309     20.237      0.000       5.648       6.860
Sabado               2.2302      0.307      7.256      0.000       1.628       2.833
Hora_1               0.1759      0.562      0.313      0.754      -0.925       1.277
Hora_2              -2.0699      0.564     -3.672      0.000      -3.175      -0.965
Hora_3              -3.9052      0.566     -6.904      0.000      -5.014      -2.797
Hora_4              -5.0109      0.567     -8.844      0.000      -6.122      -3.900
Hora_5              -5.4978      0.567     -9.701      0.000      -6.609      -4.387
Hora_6              -4.3142      0.566     -7.627      0.000      -5.423      -3.206
Hora_7              -1.9316      0.562     -3.434      0.001      -3.034      -0.829
Hora_8               0.5078      0.561      0.905      0.365      -0.592       1.607
Hora_9               1.6959      0.561      3.021      0.003       0.596       2.796
Hora_10              1.4629      0.562      2.605      0.009       0.362       2.564
Hora_11             -0.2714      0.561     -0.483      0.629      -1.372       0.829
Hora_12             -1.5585      0.561     -2.777      0.005      -2.658      -0.459
Hora_13             -1.9132      0.561     -3.410      0.001      -3.013      -0.814
Hora_14             -2.6416      0.561     -4.709      0.000      -3.741      -1.542
Hora_15             -3.8088      0.561     -6.789      0.000      -4.908      -2.709
Hora_16             -5.2017      0.561     -9.264      0.000      -6.302      -4.101
Hora_17             -4.9321      0.562     -8.784      0.000      -6.033      -3.832
Hora_18             -2.2176      0.561     -3.953      0.000      -3.317      -1.118
Hora_19              0.8638      0.561      1.539      0.124      -0.236       1.964
Hora_20              3.5728      0.562      6.353      0.000       2.470       4.675
Hora_21              4.2428      0.565      7.510      0.000       3.135       5.350
Hora_22              3.5003      0.566      6.188      0.000       2.392       4.609
Hora_23              1.9923      0.562      3.542      0.000       0.890       3.095
Festivo_Nacional    -0.8944      0.498     -1.795      0.073      -1.871       0.082
MIBGAS_FW1D          2.1483      0.004    530.269      0.000       2.140       2.156
=======================

In [17]:
db.loc[:, "lag_24"] = db.Spot_electricidad.shift(24)
db = db.dropna()

### Backtesting regresión lineal

In [18]:
def bascktesting_regresion_lineal(db, año, mes, rolling_window, days_start):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Spot_electricidad')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)

            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Spot_electricidad')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date
    
pred, real, dates = bascktesting_regresion_lineal(db.loc["2019":"2021-04"], 2020, 1, rolling_window = True, days_start = 150)

In [5]:
# Backtesting rapido

# for dias in [7, 14, 21, 30, 60, 90, 150, 220, 365]:
#     print(dias)
#     pred, real, dates = bascktesting_regresion_lineal(db.loc["2019":"2021-04"], 2020, 1, rolling_window = True, days_start = dias)
#     resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
#     calculo_metricas(resultados)

### Cálculo de las métricas

In [19]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)

def calculo_metricas(dataframe):
    dataframe.loc[:, "MAE"] = mae(dataframe.Pred, dataframe.Real)
    dataframe.loc[:, "MAPE"] = mape(dataframe.Pred, dataframe.Real)
    
    for metrica in ["MAE", "MAPE"]:
        if metrica == "MAE":
            print("Media ", metrica, "en el periodo:", round(float(dataframe.loc["2020-01":"2021-04"][[metrica]].mean()), 3), '€/MWh\nDesviacion estandar del ', metrica,':' ,round(float(dataframe.loc["2020-01":"2021-04"][[metrica]].std()), 3), "€/MWh\n")
        else:
            print("Media ", metrica, "en el periodo:", round(float(dataframe.loc["2020-01":"2021-04"][[metrica]].mean()), 3), '%\nDesviacion estandar del ', metrica,':' ,round(float(dataframe.loc["2020-01":"2021-04"][[metrica]].std()), 3), "%\n")
    
    print('El RMSE para el periodo es:', round(rmse(dataframe.Pred, dataframe.Real),3), "€/MWh")
    
    metricas_mensuales_dataframe = dataframe.loc["2020-01":"2021-04"].groupby([dataframe.loc["2020-01":"2021-04"].index.year, dataframe.loc["2020-01":"2021-04"].index.month])[["MAE", "MAPE"]].mean()
    up_down_price = dataframe[["Pred", "Real"]].diff(1).dropna()
#     up_down_price = up_down_price[up_down_price.loc[:, "Real"] != 0.00]
    up_down_price.loc[:, "Ind_Pred"] = [1 if data > 0 else 0 for data in up_down_price.Pred]
    up_down_price.loc[:, "Ind_Real"] = [1 if data > 0 else 0 for data in up_down_price.Real]
    up_down_price.loc[:, "Aciertos"] = up_down_price.loc[:, "Ind_Pred"] == up_down_price.loc[:, "Ind_Real"]
    print('Porcentaje de acierto tendencias:', round(up_down_price.Aciertos.value_counts()[1] / up_down_price.Aciertos.value_counts().sum() * 100, 2))
    return dataframe, metricas_mensuales_dataframe

reg_lineal, reg_lineal_metricas_mensuales = calculo_metricas(resultados)

Media  MAE en el periodo: 5.978 €/MWh
Desviacion estandar del  MAE : 5.827 €/MWh

Media  MAPE en el periodo: 58.183 %
Desviacion estandar del  MAPE : 543.753 %

El RMSE para el periodo es: 8.348 €/MWh
Porcentaje de acierto tendencias: 80.13


In [21]:
reg_lineal.to_csv('Resultados/resultados_reg_lineal.csv')

In [20]:
import plotly.graph_objects as go

def graficas(dataframe):
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dataframe.loc["2019-01":].index, y=dataframe.loc["2019-01":].Real,
                        mode='lines',
                        name='Real')).add_trace(go.Scatter(x=dataframe.loc["2019-01":].index, y=dataframe.loc["2019-01":].Pred,
                        mode='lines',
                        name='Predicciones'))
    
    fig.show()
    
    # Create traces
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=dataframe.loc["2019-07"].index, y=dataframe.loc["2019-07"].Real,
#                     mode='lines',
#                     name='Real')).add_trace(go.Scatter(x=dataframe.loc["2019-07"].index, y=dataframe.loc["2019-07"].Pred,
#                     mode='lines',
#                     name='Predicciones'))

#     fig.show()
    
graficas(reg_lineal)